This notebook is a implementation of LightGCN model for collaborating filtering. The model is based on the paper [LightGCN: Simplifying and Powering Graph Convolution Network for Recommendation](https://arxiv.org/abs/2002.02126) by Xiangnan He, Kuan Deng, Xiang Wang, Yan Li, Yongdong Zhang, Meng Wang.

In [142]:
# Standard library imports
import random
import time

# Third-party imports
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import degree

from tqdm.notebook import tqdm
from sklearn import preprocessing as pp

In [144]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = 'cpu'

### Data Importation

In [145]:
columns_name=['user_id','item_id','rating']
df = pd.read_csv('./drive/MyDrive/Colab/NML/data/ratings_train.csv', names=columns_name, header=0)
print(len(df))
df.dropna(inplace=True)
display(df.head(5))

4781183


,user_id,item_id,rating
0,2,4081,4
1,2,260,5
2,2,2318,3
3,2,26,4
4,2,315,3


Because the input data is too large we keep the ratings given by the first **30000 users** only for the positive ratings, we can keep all the ratings for the negative ratings as they are less than the positive ratings.

In [146]:
df_pos = df[df['user_id'] < 30000] 

We split the ratings into **good** and **bad** ratings with a threshold of 3 on the scale of ratings from 1 to 5.

In [ ]:
df = df[df['rating'] < 3]
df_pos = df_pos[df_pos['rating'] >= 3]

In [149]:
# What's the distribution of badly rated movies?
print("Rating Distribution")
df.groupby(['rating'])['rating'].count()

Rating Distribution


rating
1     99633
2    287396
Name: rating, dtype: int64

We put this ratings to train positive and negative models.

We also import the test dataset of ratings that will be used to evaluate the models.

In [150]:
train_df = df
train_df_pos = df_pos
test_df = pd.read_csv('./drive/MyDrive/Colab/NML/data/ratings_test.csv', names=columns_name, header=0)

In [151]:
print("Train Size  : ", len(train_df))
print("Test Size : ", len (test_df))

Train Size  :  387029
Test Size :  1195296


Since we performed the train/test randomly on the interactions, not all users and items may be present in the training sets. We will relabel all of users and items to ensure the highest label is the number of users and items, respectively.

In [152]:
le_user = pp.LabelEncoder()
le_item = pp.LabelEncoder()
le_user_pos = pp.LabelEncoder()
le_item_pos = pp.LabelEncoder()
train_df['user_id_idx'] = le_user.fit_transform(train_df['user_id'].values)
train_df['item_id_idx'] = le_item.fit_transform(train_df['item_id'].values)
train_df_pos['user_id_idx'] = le_user_pos.fit_transform(train_df_pos['user_id'].values)
train_df_pos['item_id_idx'] = le_item_pos.fit_transform(train_df_pos['item_id'].values)

Then we keep only the users and items that are present in the training set for the test set.

In [153]:
train_user_ids = train_df['user_id'].unique()
train_item_ids = train_df['item_id'].unique()

train_user_ids_pos = train_df_pos['user_id'].unique()
train_item_ids_pos = train_df_pos['item_id'].unique()

print(len(train_user_ids), len(train_item_ids))

test_df = test_df[
  (test_df['user_id'].isin(train_user_ids)) & \
  (test_df['item_id'].isin(train_item_ids)) & \
  (test_df['user_id'].isin(train_user_ids_pos)) & \
  (test_df['item_id'].isin(train_item_ids_pos))
]
print(len(test_df))

46334 9920
618861


In [154]:
test_df['user_id_idx'] = le_user.transform(test_df['user_id'].values)
test_df['item_id_idx'] = le_item.transform(test_df['item_id'].values)

In [155]:
n_users = train_df['user_id_idx'].nunique()
n_items = train_df['item_id_idx'].nunique()
n_users_pos = train_df_pos['user_id_idx'].nunique()
n_items_pos = train_df_pos['item_id_idx'].nunique()
print("Number of Unique Users : ", n_users)
print("Number of unique Items : ", n_items)

Number of Unique Users :  46334
Number of unique Items :  9920


We need to define the notion of positive and negative edges: `positive edges` are those that exist in the graph and `negative edges` are those that don’t. In our bipartite graph, we can define for user u the set of all positive edges containing u, E(u) and the set of all negative edges containing u, E_neg(u).

In [156]:
def data_loader(data, batch_size, n_usr, n_itm):

    def sample_neg(x):
        while True:
            neg_id = random.randint(0, n_itm - 1)
            if neg_id not in x:
                return neg_id

    interected_items_df = data.groupby('user_id_idx')['item_id_idx'].apply(list).reset_index()
    indices = [x for x in range(n_usr)]

    if n_usr < batch_size:
        users = [random.choice(indices) for _ in range(batch_size)]
    else:
        users = random.sample(indices, batch_size)
    users.sort()
    users_df = pd.DataFrame(users,columns = ['users'])

    interected_items_df = pd.merge(interected_items_df, users_df, how = 'right', left_on = 'user_id_idx', right_on = 'users')
    pos_items = interected_items_df['item_id_idx'].apply(lambda x : random.choice(x)).values
    neg_items = interected_items_df['item_id_idx'].apply(lambda x: sample_neg(x)).values

    return (
        torch.LongTensor(list(users)).to(device),
        torch.LongTensor(list(pos_items)).to(device) + n_usr,
        torch.LongTensor(list(neg_items)).to(device) + n_usr
    )

data_loader(train_df, 16, n_users, n_items)

(tensor([   54,  5504,  8609, 13425, 13711, 16523, 21233, 22976, 25007, 29993,
         34542, 38208, 39803, 40889, 41329, 41991]),
 tensor([46359, 55735, 50415, 54228, 46373, 53006, 47745, 47397, 48375, 46541,
         46355, 49148, 50049, 46981, 46843, 46336]),
 tensor([54769, 55895, 53351, 47195, 46392, 53826, 49875, 48666, 53383, 53586,
         46736, 50124, 55747, 51199, 48604, 54088]))

## Edge Index

PyG represents graphs as sparse lists of node pairs. Since our graph is undirected, we need to include each edge twice, once for the edges from the users to the items and vice-versa.

Similar to above, we add `n_users` to the item tensor to ensure that every node in the graph has a unique identifier.

In [157]:
array_tr_user = np.array(train_df.user_id_idx)
array_tr_item = np.array(train_df.item_id_idx)
u_t = torch.LongTensor(array_tr_user)
i_t = torch.LongTensor(array_tr_item) + n_users

array_tr_user_pos = np.array(train_df_pos.user_id_idx)
array_tr_item_pos = np.array(train_df_pos.item_id_idx)
u_t_pos = torch.LongTensor(array_tr_user_pos)
i_t_pos = torch.LongTensor(array_tr_item_pos) + n_users_pos

train_edge_index = torch.stack((
  torch.cat([u_t, i_t]),
  torch.cat([i_t, u_t])
)).to(device)

train_edge_index_pos = torch.stack((
  torch.cat([u_t_pos, i_t_pos]),
  torch.cat([i_t_pos, u_t_pos])
)).to(device)

Let's confirm that the first and last edges match the middle two edges, but with the order of nodes swapped.

In [158]:
train_edge_index[:,-1], train_edge_index[:, 0]

(tensor([54440, 43722]), tensor([    3, 46588]))

In [159]:
train_edge_index[:, len(train_df)-1], train_edge_index[:, len(train_df)]

(tensor([43722, 54440]), tensor([46588,     3]))

### LightGCN Convolutional Layer

The LightGCN architecture is governed by the following rules:

$$e_{u}^{(k+1)} = \sum\limits_{i \in N_u} \frac{1}{\sqrt{|N_u|}\sqrt{|N_i|}}e^{(k)}_i$$

$$e_{i}^{(k+1)} = \sum\limits_{u \in N_i} \frac{1}{\sqrt{|N_i|}\sqrt{|N_u|}}e^{(k)}_u$$
In essence, the embedding for each node after a single LightGCN layer is the sum of the synthetic normalized embeddings of it's neighbors before the layer.

In [160]:
class LightGCNConv(MessagePassing):
  def __init__(self, **kwargs):
    super().__init__(aggr='add')

  def forward(self, x, edge_index):
    # Compute normalization
    from_, to_ = edge_index
    deg = degree(to_, x.size(0), dtype=x.dtype)
    deg_inv_sqrt = deg.pow(-0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
    norm = deg_inv_sqrt[from_] * deg_inv_sqrt[to_]

    # Start propagating messages (no update after aggregation)
    return self.propagate(edge_index, x=x, norm=norm)

  def message(self, x_j, norm):
    return norm.view(-1, 1) * x_j

In [ ]:
class RecSysGNN(nn.Module):
  def __init__(
      self,
      latent_dim,
      num_layers,
      num_users,
      num_items,
      model, # 'LightGCN'
  ):
    super(RecSysGNN, self).__init__()

    self.model = model
    self.embedding = nn.Embedding(num_users + num_items, latent_dim)

    self.convs = nn.ModuleList(LightGCNConv() for _ in range(num_layers))

    self.init_parameters()


  def init_parameters(self):
    # Authors of LightGCN report higher results with normal initialization
    nn.init.normal_(self.embedding.weight, std=0.1)

  def forward(self, edge_index):

    emb0 = self.embedding.weight
    embs = [emb0]

    emb = emb0
    for conv in self.convs:
      emb = conv(x=emb, edge_index=edge_index)
      print('emb size conv', emb.shape)
      embs.append(emb)

    out = (torch.mean(torch.stack(embs, dim=0), dim=0))

    return emb0, out


  def encode_minibatch(self, users, pos_items, neg_items, edge_index):
    emb0, out = self(edge_index)
    return (
        out[users],
        out[pos_items],
        out[neg_items],
        emb0[users],
        emb0[pos_items],
        emb0[neg_items]
    )

## Loss function and metrics

We implement both the Bayesian Personalized Ranking loss function for a single minibatch of users, positive items, and negative items, as well as the precision K and recall K metrics.

In [164]:
def compute_bpr_loss(users, users_emb, pos_emb, neg_emb, user_emb0,  pos_emb0, neg_emb0):
  # compute loss from initial embeddings, used for regulization
  reg_loss = (1 / 2) * (
    user_emb0.norm().pow(2) +
    pos_emb0.norm().pow(2)  +
    neg_emb0.norm().pow(2)
  ) / float(len(users))

  # compute BPR loss from user, positive item, and negative item embeddings
  pos_scores = torch.mul(users_emb, pos_emb).sum(dim=1)
  neg_scores = torch.mul(users_emb, neg_emb).sum(dim=1)

  bpr_loss = torch.mean(F.softplus(neg_scores - pos_scores))

  return bpr_loss, reg_loss

In [165]:
def get_metrics_pos(user_Embed_wts_pos, item_Embed_wts_pos, n_users_pos, n_items_pos, train_data, test_data, K):
  test_user_ids = torch.LongTensor(test_data['user_id_idx'].unique())
  # compute the score of all user-item pairs
  relevance_score = torch.matmul(user_Embed_wts_pos, torch.transpose(item_Embed_wts_pos,0, 1))

  # create dense tensor of all user-item interactions
  i = torch.stack((
    torch.LongTensor(train_data['user_id_idx'].values),
    torch.LongTensor(train_data['item_id_idx'].values)
  ))
  v = torch.ones((len(train_data)), dtype=torch.float64)
  interactions_t = torch.sparse.FloatTensor(i, v, (n_users_pos, n_items_pos))\
      .to_dense().to(device)

  # mask out training user-item interactions from metric computation
  relevance_score = torch.mul(relevance_score, (1 - interactions_t))

  # compute top scoring items for each user
  topk_relevance_indices = torch.topk(relevance_score, K).indices
  topk_relevance_indices_df = pd.DataFrame(topk_relevance_indices.cpu().numpy(),columns =['top_indx_'+str(x+1) for x in range(K)])
  topk_relevance_indices_df['user_ID'] = topk_relevance_indices_df.index
  topk_relevance_indices_df['top_rlvnt_itm'] = topk_relevance_indices_df[['top_indx_'+str(x+1) for x in range(K)]].values.tolist()
  topk_relevance_indices_df = topk_relevance_indices_df[['user_ID','top_rlvnt_itm']]

  test_data_sup3 = test_data[test_data['rating'] >= 3]
  test_interacted_items_sup3 = test_data_sup3.groupby('user_id_idx')['item_id_idx'].apply(list).reset_index()

  test_data_inf3 = test_data[test_data['rating'] < 3]
  test_interacted_items_inf3 = test_data_inf3.groupby('user_id_idx')['item_id_idx'].apply(list).reset_index()

  # Ensure all users are included in both TP and FP calculations
  all_users = pd.DataFrame({'user_id_idx': test_df['user_id_idx'].unique()})

  # Merge with all_users to include all users
  test_interacted_items_sup3 = pd.merge(all_users, test_interacted_items_sup3, on='user_id_idx', how='left')
  test_interacted_items_inf3 = pd.merge(all_users, test_interacted_items_inf3, on='user_id_idx', how='left')

  # Fill NaN values with empty lists
  test_interacted_items_sup3['item_id_idx'] = test_interacted_items_sup3['item_id_idx'].apply(lambda x: x if isinstance(x, list) else [])
  test_interacted_items_inf3['item_id_idx'] = test_interacted_items_inf3['item_id_idx'].apply(lambda x: x if isinstance(x, list) else [])

  # True Positives
  TP = pd.merge(test_interacted_items_sup3, topk_relevance_indices_df, how='left', left_on='user_id_idx', right_on='user_ID')
  TP['intrsctn_itm'] = [list(set(a).intersection(b)) for a, b in zip(TP.item_id_idx, TP.top_rlvnt_itm)]

  # False Positives
  FP = pd.merge(test_interacted_items_inf3, topk_relevance_indices_df, how='left', left_on='user_id_idx', right_on='user_ID')
  FP['intrsctn_itm'] = [list(set(a).intersection(b)) for a, b in zip(FP.item_id_idx, FP.top_rlvnt_itm)]

  return TP, FP

In [247]:
def get_metrics_neg(user_Embed_wts, item_Embed_wts, n_users, n_items, train_data, test_data, K):
  test_user_ids = torch.LongTensor(test_data['user_id_idx'].unique())
  # compute the score of all user-item pairs
  relevance_score = torch.matmul(user_Embed_wts, torch.transpose(item_Embed_wts,0, 1))

  # create dense tensor of all user-item interactions
  i = torch.stack((
    torch.LongTensor(train_df['user_id_idx'].values),
    torch.LongTensor(train_df['item_id_idx'].values)
  ))
  v = torch.ones((len(train_df)), dtype=torch.float64)
  interactions_t = torch.sparse.FloatTensor(i, v, (n_users, n_items))\
      .to_dense().to(device)

  # mask out training user-item interactions from metric computation
  relevance_score = torch.mul(relevance_score, (1 - interactions_t))

  # compute top scoring items for each user
  topk_relevance_indices = torch.topk(relevance_score, K).indices
  topk_relevance_indices_df = pd.DataFrame(topk_relevance_indices.cpu().numpy(),columns =['top_indx_'+str(x+1) for x in range(K)])
  topk_relevance_indices_df['user_ID'] = topk_relevance_indices_df.index
  topk_relevance_indices_df['top_rlvnt_itm'] = topk_relevance_indices_df[['top_indx_'+str(x+1) for x in range(K)]].values.tolist()
  topk_relevance_indices_df = topk_relevance_indices_df[['user_ID','top_rlvnt_itm']]

  test_data_sup3 = test_data[test_data['rating'] > 3]
  test_interacted_items_sup3 = test_data_sup3.groupby('user_id_idx')['item_id_idx'].apply(list).reset_index()

  # Ensure all users are included in both TP and FP calculations
  all_users = pd.DataFrame({'user_id_idx': test_data['user_id_idx'].unique()})

  # Merge with all_users to include all users
  test_interacted_items_sup3 = pd.merge(all_users, test_interacted_items_sup3, on='user_id_idx', how='left')

  # Fill NaN values with empty lists
  test_interacted_items_sup3['item_id_idx'] = test_interacted_items_sup3['item_id_idx'].apply(lambda x: x if isinstance(x, list) else [])

  # False Negatives
  FN = pd.merge(test_interacted_items_sup3, topk_relevance_indices_df, how='left', left_on='user_id_idx', right_on='user_ID')
  FN['intrsctn_itm'] = [list(set(a).intersection(b)) for a, b in zip(FN.item_id_idx, FN.top_rlvnt_itm)]

  return FN


In [258]:
def get_recall_accuracy(TP, FP, FN):
    # Ensure TP, FP, and FN are lists of lists
    assert all(isinstance(i, list) for i in TP), "TP must be a list of lists"
    assert all(isinstance(i, list) for i in FP), "FP must be a list of lists"
    assert all(isinstance(i, list) for i in FN), "FN must be a list of lists"

    # Calculate recall for each user
    recall = [
        len(TP[i]) / (len(TP[i]) + len(FN[i])) if (len(TP[i]) + len(FN[i])) > 0 else 0
        for i in range(len(TP))
    ]

    # Calculate precision for each user
    precision = [
        len(TP[i]) / (len(TP[i]) + len(FP[i])) if (len(TP[i]) + len(FP[i])) > 0 else 0
        for i in range(len(TP))
    ]

    return np.mean(recall), np.mean(precision)

## Train and evaluate models

Now that we've implemented both LightGCN and NGCF in PyG, we can train and evaluate their performance!

In [168]:
latent_dim = 64
n_layers = 3

EPOCHS = 10
BATCH_SIZE = 4096
DECAY = 0.0001
LR = 0.005
K = 20

In [169]:
def train_and_eval(model, optimizer, train_df):

  for epoch in tqdm(range(EPOCHS)):
      n_batch = int(len(train_df)/BATCH_SIZE)

      final_loss_list = []
      bpr_loss_list = []
      reg_loss_list = []

      model.train()
      for batch_idx in tqdm(range(n_batch)):

          optimizer.zero_grad()

          users, pos_items, neg_items = data_loader(train_df, BATCH_SIZE, n_users, n_items)
          users_emb, pos_emb, neg_emb, userEmb0,  posEmb0, negEmb0 = model.encode_minibatch(users, pos_items, neg_items, train_edge_index)

          bpr_loss, reg_loss = compute_bpr_loss(
            users, users_emb, pos_emb, neg_emb, userEmb0,  posEmb0, negEmb0
          )
          reg_loss = DECAY * reg_loss
          final_loss = bpr_loss + reg_loss

          final_loss.backward()
          optimizer.step()

          final_loss_list.append(final_loss.item())
          bpr_loss_list.append(bpr_loss.item())
          reg_loss_list.append(reg_loss.item())

      torch.save(model.state_dict(), f'/content/drive/MyDrive/Colab/NML/model_epoch{epoch}_2.pth')


### Train and eval LightGCN

We train two LightGCN models, one for positive ratings and one for negative ratings. We then evaluate the performance of the models on the test set.

In [170]:
lightgcn_neg = RecSysGNN(
  latent_dim=latent_dim,
  num_layers=n_layers,
  num_users=n_users,
  num_items=n_items,
  model='LightGCN'
)

lightgcn_neg.load_state_dict(torch.load('/content/drive/MyDrive/Colab/NML/model_neg.pth',  map_location=device))
lightgcn_neg.to(device)

# Train the model
optimizer = torch.optim.Adam(lightgcn_neg.parameters(), lr=LR)
print("Size of Learnable Embedding : ", [x.shape for x in list(lightgcn_neg.parameters())])
train_and_eval(lightgcn_neg, optimizer, train_df)

Size of Learnable Embedding :  [torch.Size([56254, 64])]


In [172]:
lightgcn_pos = RecSysGNN(
  latent_dim=latent_dim,
  num_layers=n_layers,
  num_users=n_users_pos,
  num_items=n_items_pos,
  model='LightGCN'
)

lightgcn_pos.load_state_dict(torch.load('/content/drive/MyDrive/Colab/NML/model_pos.pth',  map_location=device))
lightgcn_pos.to(device)

RecSysGNN(
  (embedding): Embedding(39998, 64)
  (convs): ModuleList(
    (0-2): 3 x LightGCNConv()
  )
)

To choose the best K we compute the median of ratings for positive and negatives ratings and we take them as `K_pos` and `K_neg`. 

In [268]:
groups_neg = df.groupby('user_id_idx')['item_id_idx'].apply(list)
groups_neg = [len(x) for x in groups_neg]
K_neg = np.median(groups_neg).astype(int)
print('K_neg :', K_neg)

K_neg : 6


In [269]:
groups_pos = df_pos.groupby('user_id_idx')['item_id_idx'].apply(list)
groups_pos = [len(x) for x in groups_pos]
K_pos = np.median(groups_pos).astype(int)
print('K_pos :', K_pos)

K_pos : 83


Now calculate the precision and recall for the sum of the positive and negative ratings.

In [270]:
# Evaluate the positive model
_, out = lightgcn_pos(train_edge_index_pos)
final_user_Embed, final_item_Embed = torch.split(out, (n_users_pos, n_items_pos))
TP,FP= get_metrics_pos( final_user_Embed, final_item_Embed, n_users_pos, n_items_pos, train_df_pos, test_df, K_pos)

# Evaluate the negative model
_, out = lightgcn_neg(train_edge_index)
final_user_Embed, final_item_Embed = torch.split(out, (n_users, n_items))
FN= get_metrics_neg( final_user_Embed, final_item_Embed, n_users, n_items, train_df, test_df, K_neg)

Now to calculate the precision and recall for the sum of the positive and negative ratings, we need to find the users that are present in the two training sets and the test set, to calculate the precision and recall for each user. Then we calculate the average precision and recall for all users.

In [271]:
TP.rename(columns={'intrsctn_itm': 'TP'}, inplace=True)
FP.rename(columns={'intrsctn_itm': 'FP'}, inplace=True)
FN.rename(columns={'intrsctn_itm': 'FN'}, inplace=True)

merged_df = pd.merge(TP, FP, on='user_ID', how='inner')
merged_df = pd.merge(merged_df, FN, on='user_ID', how='inner')

TP = merged_df['TP']
FP = merged_df['FP']
FN = merged_df['FN']


In [272]:
recall, precision = get_recall_accuracy(TP,FP,FN)
print(recall)
print(precision)

0.6044794789533449
0.6088721166190417


We can see that the recall and precision are about the same, but are not really high. We could train on more epoch to improve the results.

this method is inspire by the blog page: 

https://medium.com/stanford-cs224w/recommender-systems-with-gnns-in-pyg-d8301178e377

Paper References

    Harper, F. Maxwell, and Konstan, Joseph A. “The MovieLensDatasets: History and Context.” ACM Transactions on Interactive Intelligence Systems (TiiS) 5, 4. 2015.

    He, Xiangnan, et al. “LightGCN: Simplifying and powering graph convolution network for recommendation.” Proceedings of the 43rd International ACM SIGIR Conference on Research and Development in Information Retrieval. 2020.
    
    Wang, Xiang, et al. “Neural graph collaborative filtering.” Proceedings of the 42nd International ACM SIGIR Conference on Research and Development in Information Retrieval. 2019.